In [ ]:
!pip install boto3
!pip install langchain
!pip install langchain_aws

# Module 1, Activity 2: Modifying a Simple Chain

In this notebook we will learn about how to create chains in LangChain and make them more sophisticated through the use of prompt templates and memory.

In [ ]:
import json
from pprint import pprint

from langchain.chains import LLMChain
from langchain_aws import ChatBedrock
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableLambda

In [ ]:
AWS_DEFAULT_REGION="..."
AWS_ACCESS_KEY_ID="..."
AWS_SECRET_ACCESS_KEY="..."
AWS_SESSION_TOKEN="..."

## Establishing our connection to Bedrock

Be sure to pick a model that is already loaded into the Bedrock.  Have fun adjusting `temperature` and `max_tokens.`

In [ ]:
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # Replace with your desired model ID
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    temperature=0.5,
    max_tokens=200,
)

## Prompts and Prompt Templates

In LangChain, **system prompts** (provide the foundational instructions and context for the AI model.  They define the model's role, behavior, and any specific output format or constraints, ensuring that all subsequent responses align with the intended purpose.  In contrast, **human prompts** represent the dynamic, user-supplied inputs that drive the conversation or query.  They capture the specific question or instruction that the user wants the model to address.  The `ChatPromptTemplate` acts as a framework that seamlessly integrates both types of prompts into a coherent message sequence.  By combining system and human messages, the `ChatPromptTemplate` creates a structured dialogue that guides the model to generate responses that are both contextually relevant and properly formatted.

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant.  Return your answer in pirate speak."
)
human_prompt = HumanMessagePromptTemplate.from_template("{input}")
prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

## Chains

Chains in LangChain are modular sequences that connect various components—such as prompt templates, language models, and output parsers—into a unified workflow.  They enable developers to construct complex, multi-step processing pipelines where each step transforms or utilizes the output from the previous one.  LCEL (LangChain Chain Expression Language) implements this concept using a concise, pipe operator (|) syntax that clearly defines the data flow between these components.  By leveraging LCEL, you can declaratively compose chains that are both flexible and readable, allowing for rapid prototyping and iterative development of sophisticated AI-powered applications.

In this code block we are creating a simple, two-part chain with our prompt and our LLM.

In [ ]:
chain = prompt | llm
chain.invoke("What is the recipe for mayonnaise?")

## Output Parsers

The `AIMessage` format might not be the best way for the output of your LLM calll to be rendered.  So we can keep adding to the chain by adding another component: `StrOutputParser`.  It converts a language model's output into a clean, plain text string by stripping away any additional metadata or formatting.  So we can see in this example that we just add it modularly to the end of our chain.

In [ ]:
chain = prompt | llm | StrOutputParser()
chain.invoke("What is the recipe for mayonnaise?")

## Other Output Parsers

There are several different output parsers available in LangChain.  You can read about them all in the [API docs](https://python.langchain.com/api_reference/core/output_parsers.html).  Here is another common use case using the `JsonOutputParser` to get the result of the LLM call in a convenient JSON format.  Note that we changed the system prompt to reflect how we wanted that JSON to be formatted.

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. Return your answer in JSON format with exactly two keys: 'response' and 'details'."
)
human_prompt = HumanMessagePromptTemplate.from_template("{input}")
prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

chain = prompt | llm | JsonOutputParser()

json_output = chain.invoke("What is the recipe for mayonnaise?")
pprint(json_output)

In [ ]:
chain.invoke("What recipe did I just ask you for?")

## Oops!

Notice that we just asked the LLM to tell us about what we have been talking about with it.  However, LLM's do not, by default, have any memory.  This is something we need to add to it by creating a chat history.

_**Note:**_

It is likely when you run the below cell you will get a deprecation error.  This is because LangChain is changing how they handle conversational memory, encouraging migration to their new platform, LangGraph.  LangGraph is a sophisticated platform used for the orchestration of multi-agent bots that use tools.  At this stage it is beyond the scope of where we are in this workshop but we will discuss it when we get to "Module 4: Agents and Tool Use."  

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

## A node about this function

We will now be creating a chain through a different approach than LCEL, which will allow us to add this memory we just created.  However, the output of that is in a different format than the chain created above with LCEL.  So this helper function will be used to convert it back into something LCEL can work with.

In [ ]:
def convert_output_to_str(x):

    """
    Recursively converts the input object into a JSON-serializable string.

    This function processes the input recursively:
    - If the object is a string, it is returned as is.
    - If the object has a 'content' attribute (e.g., a HumanMessage), it retrieves the content and processes it recursively.
    - If the object is a dictionary, it recursively converts each key-value pair.
    - If the object is a list, it recursively converts each element in the list.
    - For any other object types, it returns the object unchanged.

    After recursive conversion, if the resulting object is a dictionary or list,
    it is serialized into a JSON string using `json.dumps`. Otherwise, its string
    representation is returned.

    This ensures that the output passed to the downstream parser (e.g., StrOutputParser)
    is a valid JSON-serializable string.
    """
    def recursive_convert(obj):
        # If already a string, return it.
        if isinstance(obj, str):
            return obj
        # If it's a message, use its content.
        if hasattr(obj, "content"):
            return recursive_convert(obj.content)
        # If it's a dict, process each key-value pair.
        if isinstance(obj, dict):
            return {k: recursive_convert(v) for k, v in obj.items()}
        # If it's a list, process each element.
        if isinstance(obj, list):
            return [recursive_convert(item) for item in obj]
        # Otherwise, just return the object.
        return obj

    converted = recursive_convert(x)
    # If the result is a dict or list, return a JSON string.
    if isinstance(converted, (dict, list)):
        return json.dumps(converted)
    return str(converted)

convert_to_str = RunnableLambda(convert_output_to_str)

## Adding Memory to Chains

We can't quite add the memory in by itself to the existing chain because the LCEL pipe composition creates a new chain that does not carry over the memory state from your original LLMChain. To get multi-turn behavior where the chain remembers previous inputs, we create the chain with `LLMChain()`, which allows us to add the memory in directly.

Also note that we have to then add `{chat_history}` to the prompt so the information contained in memory makes its way back into the system.  However, this has some drawbacks that we will discuss shortly!

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. Use the conversation history to inform your answers. Conversation history: {chat_history}"
)
human_prompt = HumanMessagePromptTemplate.from_template("{input}")
prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

chain_with_memory = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
)

response = chain_with_memory.invoke("What is the recipe for mayonnaise?")
print(response['text'])

In [ ]:
response = chain_with_memory.invoke("What recipe did I just ask you for?")
print(response['text'])

## Final Comments

You might have realized that adding the chat history into the system prompt might become a problem down the road as our conversations with our bot get longer and longer.  This is because your prompt will keep increasing in token size until you either hit the token limit, wind up with a really big LLM bill, or both!  So it is important to think about strategies for only providing a limited amount of information in the history.  You can read about some strategies for limiting the history size [here](https://python.langchain.com/docs/tutorials/chatbot/#managing-conversation-history).